In [0]:
!pip install torch

    100% |████████████████████████████████| 591.8MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x6170a000 @  0x7f80f04f22a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
!pip install h5py

In [0]:
!pip install torchnet

    100% |████████████████████████████████| 256kB 24.0MB/s 
    100% |████████████████████████████████| 204kB 23.7MB/s 
  Running setup.py bdist_wheel for torchnet ... - \ done
  Stored in directory: /root/.cache/pip/wheels/e1/03/fb/1c212c2f20905cdf97fe39022946cf16b8e66ed754a6663400
  Running setup.py bdist_wheel for visdom ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fb/ef/d8/ad95fbafb505c507b476c4e907af570d2dd41ae725cdc9b391
  Running setup.py bdist_wheel for torchfile ... - done
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchnet visdom torchfile


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install torchtext==0.2.3


    100% |████████████████████████████████| 51kB 2.5MB/s 
  Running setup.py bdist_wheel for torchtext ... - done
  Stored in directory: /root/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext


In [0]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [0]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD

import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.BytbeTensor


use cuda: True


In [0]:
nn = nn.cuda()

In [0]:
type(LongTensor)

torch.tensortype

In [0]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('gdrive/My Drive/datasets/ML/all (3)', subset='train')
valSet = Dataset('gdrive/My Drive/datasets/ML/all (3)', subset='val')
tstSet = Dataset('gdrive/My Drive/datasets/ML/all (3)', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

# 1 To DO:
### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.

In [0]:
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = torch.nn.Linear(75, 100, 1)  #  linear layer that can map features of 75-dimension to 100-dimension.
        self.recurrent_layer = torch.nn.LSTM(100, 100,1)  # Layer LSTM layer with hidden size of 100
        self.classify_layer = torch.nn.Linear(100, 10)    # Linear layer that goes from 100 to num_class (10).
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        #print(input.shape)
        inter = self.project_layer(input)
        #print((inter.shape,type(inter)))
        rnn_outputs, (hn, cn) = self.recurrent_layer(inter)
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## 2 To Do:
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [0]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)#None
criterion = torch.nn.CrossEntropyLoss()#None

In [0]:
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.item())
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1000
evaluate_every_epoch = 1
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


In [0]:
type(model)

__main__.SequenceClassify

In [0]:
print(model)
#print(model1)

SequenceClassify(
  (project_layer): Linear(in_features=75, out_features=100, bias=True)
  (recurrent_layer): LSTM(100, 100)
  (classify_layer): Linear(in_features=100, out_features=10, bias=True)
)


## 3 To do: Model for kaggle submission

In [0]:
# class SequenceClassify(nn.Module):
#    def __init__(self):
        
#     def forward(self, input, h_t_1=None, c_t_1=None):
       
      

# model = SequenceClassify()


class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
       
        self.project_layer = torch.nn.Linear(75, 100, 1)  
        
        self.recurrent_layer = torch.nn.LSTM(75, 150,1, batch_first = True)   #batch first signifies that first dimension is sequence dimension
        
        self.recurrent_layer1 = torch.nn.LSTM(150,300,1, batch_first = True)
        
        self.recurrent_layer2 = torch.nn.LSTM(300,100,1, batch_first = True)
        
        self.downsize_layer = torch.nn.Linear(100,50)
        
        self.classify_layer = torch.nn.Linear(50, 10)    
        
        self.drop = nn.Dropout(0.2)
        
        self.bn1 = nn.BatchNorm1d(15)
        
        

    def forward(self, input, h_t_1=None, c_t_1=None):
        #inter = self.project_layer(input)
        
        rnn_outputs, (hn, cn) = self.recurrent_layer(input)
        
        rnn_outputs = self.drop(rnn_outputs)
        
        rnn_outputs, (hn,cn)= self.recurrent_layer1(rnn_outputs)
        
        #print(hn,cn)
        
        rnn_outputs = self.bn1(rnn_outputs)
        
        
        rnn_outputs = self.drop(rnn_outputs)
        
        #print(rnn_outputs.shape)
        
        rnn_outputs, (hn,cn) = self.recurrent_layer2(rnn_outputs)
        
        rnn_outputs = self.bn1(rnn_outputs)
        
        linear_outputs = self.downsize_layer(rnn_outputs[:,-1])
        
        linear_outputs = self.drop(linear_outputs)
        
        logits = self.classify_layer(linear_outputs)
        
        return logits

model = SequenceClassify()

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)#None
criterion = torch.nn.CrossEntropyLoss()#None

In [0]:
num_epochs = 1000
evaluate_every_epoch = 1
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.2951,  Accuracy: 15.55
val Epoch: 0  , Loss: 2.2948,  Accuracy: 16.00
train Epoch: 1  , Loss: 2.2950,  Accuracy: 15.45
val Epoch: 1  , Loss: 2.2945,  Accuracy: 16.50
train Epoch: 2  , Loss: 2.2946,  Accuracy: 15.85
val Epoch: 2  , Loss: 2.2942,  Accuracy: 16.25
train Epoch: 3  , Loss: 2.2935,  Accuracy: 15.85
val Epoch: 3  , Loss: 2.2938,  Accuracy: 16.50
train Epoch: 4  , Loss: 2.2943,  Accuracy: 16.30
val Epoch: 4  , Loss: 2.2934,  Accuracy: 15.75
train Epoch: 5  , Loss: 2.2925,  Accuracy: 15.80
val Epoch: 5  , Loss: 2.2930,  Accuracy: 16.75
train Epoch: 6  , Loss: 2.2933,  Accuracy: 14.35
val Epoch: 6  , Loss: 2.2926,  Accuracy: 17.50
train Epoch: 7  , Loss: 2.2917,  Accuracy: 14.80
val Epoch: 7  , Loss: 2.2920,  Accuracy: 18.00
train Epoch: 8  , Loss: 2.2911,  Accuracy: 16.95
val Epoch: 8  , Loss: 2.2915,  Accuracy: 18.00
train Epoch: 9  , Loss: 2.2909,  Accuracy: 16.05
val Epoch: 9  , Loss: 2.2910,  Accuracy: 18.25
train Epoch: 10  , Loss: 2.2904,  Accura

In [56]:
print(model)

SequenceClassify(
  (project_layer): Linear(in_features=75, out_features=100, bias=True)
  (recurrent_layer): LSTM(75, 150, batch_first=True)
  (recurrent_layer1): LSTM(150, 300, batch_first=True)
  (recurrent_layer2): LSTM(300, 100, batch_first=True)
  (downsize_layer): Linear(in_features=100, out_features=50, bias=True)
  (classify_layer): Linear(in_features=50, out_features=10, bias=True)
  (drop): Dropout(p=0.2)
  (bn1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [0]:
# Use your best model to generate results on test set.


# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results1.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(int(preds[i]))+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)
print(count)

1000


In [0]:
# num_epochs = 1000
# evaluate_every_epoch = 1
# for e in range(num_epochs):
#     run_epoch(trLD, model1, criterion1, e, True, optimizer1)
#     if e % evaluate_every_epoch == 0:
#         run_epoch(valLD, model1, criterion1, e, False, None)   


In [55]:
num_epochs = 1000
evaluate_every_epoch = 1
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.3137,  Accuracy: 9.90
val Epoch: 0  , Loss: 2.3079,  Accuracy: 10.00
train Epoch: 1  , Loss: 2.3116,  Accuracy: 10.30
val Epoch: 1  , Loss: 2.3067,  Accuracy: 10.00
train Epoch: 2  , Loss: 2.3115,  Accuracy: 9.80
val Epoch: 2  , Loss: 2.3058,  Accuracy: 10.00
train Epoch: 3  , Loss: 2.3103,  Accuracy: 9.00
val Epoch: 3  , Loss: 2.3049,  Accuracy: 10.00
train Epoch: 4  , Loss: 2.3083,  Accuracy: 9.90
val Epoch: 4  , Loss: 2.3042,  Accuracy: 10.50
train Epoch: 5  , Loss: 2.3088,  Accuracy: 9.80
val Epoch: 5  , Loss: 2.3037,  Accuracy: 9.25
train Epoch: 6  , Loss: 2.3063,  Accuracy: 9.65
val Epoch: 6  , Loss: 2.3032,  Accuracy: 12.50
train Epoch: 7  , Loss: 2.3054,  Accuracy: 9.90
val Epoch: 7  , Loss: 2.3026,  Accuracy: 15.00
train Epoch: 8  , Loss: 2.3036,  Accuracy: 10.90
val Epoch: 8  , Loss: 2.3022,  Accuracy: 14.50
train Epoch: 9  , Loss: 2.3050,  Accuracy: 10.75
val Epoch: 9  , Loss: 2.3018,  Accuracy: 14.50
train Epoch: 10  , Loss: 2.3033,  Accuracy: 10.7

In [0]:
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results1.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(int(preds[i]))+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model1, tstLD)
print(count)

1000


## 4 th to do


I have implimented a 5 layer architecture. First layer is recurrant layer with batch normalization after layer 2 and layer 3.
Model has 3 recurrant layers and 2 hidden layers
Batchnromalization is implimented at the end of every layer.

*   Layers of model:
 *  (project_layer): Linear(in_features=75, out_features=100, bias=True)
 *  (recurrent_layer): LSTM(75, 150, batch_first=True)
 * (recurrent_layer1): LSTM(150, 300, batch_first=True)
 * (recurrent_layer2): LSTM(300, 100, batch_first=True)
 * (downsize_layer): Linear(in_features=100, out_features=50, bias=True)
 * (classify_layer): Linear(in_features=50, out_features=10, bias=True)
 *  (drop): Dropout(p=0.2)
 * (bn1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 
 First LSTM layer takes 75 dimnetional input and gives the output of 150 dimentions. Second layer takes this 150 dimnetional inputs to 300 dimentional output. Third layer reduces the representation dimention . Then I have used linear 2 linear layers. I have used dropout of 0.2. I have done batch normlaization after layer two and three.

* I have ran model for 1000 epochs.
* Learning rate is same as task 2.
* Though the model is slow to converge ater epoch 300 we can see significant increase in perfomance.



